In [89]:
from collections import defaultdict
import os
import random

from datacache import get_data_dir
from mhcnames import normalize_allele_name
from mhctools import NetMHCIIpan, NetMHCIIpan4_BA

import shellinford
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, svm

from mhc2flurry.downloads import get_path
from mhc2flurry.encodable_sequences import EncodableSequences

In [4]:
train_ms_df = pd.read_csv(get_path("data_curated", "ms.by_pmid.csv.bz2"))
train_ms_df = train_ms_df.loc[
    (train_ms_df.mhc_class == "II") &
    (train_ms_df.format == "MONOALLELIC")
]
train_ms_df.head()

,pmid,sample_id,peptide,format,mhc_class,hla,expression_dataset,cell_line,intensity,original_pmid,pulldown_antibody,sample_type
0,31495665,MAPTAC_DRB1*12:01_DM-,APVKKLVVKGGKKKKQVLKFTLD,MONOALLELIC,II,HLA-DRA*01:01-DRB1*12:01,cell_line:EXPI293,EXPI293,NaN,31495665,MAPTAC,EXPI293
1,31495665,MAPTAC_DRB1*12:01_DM-,VPGPGPAPMPSDFQVLRAKY,MONOALLELIC,II,HLA-DRA*01:01-DRB1*12:01,cell_line:EXPI293,EXPI293,NaN,31495665,MAPTAC,EXPI293
2,31495665,MAPTAC_DRB1*12:01_DM-,IGNLMVSPPVKVQGKE,MONOALLELIC,II,HLA-DRA*01:01-DRB1*12:01,cell_line:EXPI293,EXPI293,NaN,31495665,MAPTAC,EXPI293
3,31495665,MAPTAC_DRB1*12:01_DM-,ALMGYATHKYLDSEEDEE,MONOALLELIC,II,HLA-DRA*01:01-DRB1*12:01,cell_line:EXPI293,EXPI293,NaN,31495665,MAPTAC,EXPI293
4,31495665,MAPTAC_DRB1*12:01_DM-,GSDQSENVDRGAGSIREA,MONOALLELIC,II,HLA-DRA*01:01-DRB1*12:01,cell_line:EXPI293,EXPI293,NaN,31495665,MAPTAC,EXPI293


In [5]:
# restrict to DRB1*01:01
use_train_ms_df = train_ms_df.loc[
    (train_ms_df.hla == 'HLA-DRA*01:01-DRB1*01:01') &
    (train_ms_df['peptide'].str.len() == 15)
].copy()

In [9]:
# the only info I need, for now, is the list of peptides that are hits. From this I'll generate a bunch of
# decoys, convert everything into encoded data, and proceed

cache_dir = get_data_dir()
fm_index_path = os.path.join(cache_dir, 'homo_sapiens_102_with_transcript_id.fm')
fm_index = shellinford.FMIndex(filename=fm_index_path)

# map each peptide to its transcript
peptide_to_doc = defaultdict(list)
# how many distinct transcripts are these from?
for peptide in use_train_ms_df.peptide:
    docs = fm_index.search(peptide)
    if not docs:
        print('Peptide not found: %s' % peptide)
    for doc in docs:
        peptide_to_doc[peptide].append(doc)

Peptide not found: VGSDWRFLRGYHQYA
Peptide not found: ETQISKTNTQTYRES
Peptide not found: YDGKDYIALKEDLRS
Peptide not found: DGKDYIALKEDLRSW
Peptide not found: APLLREVAAVRCAVH
Peptide not found: ERYIYNREEFVRFDS
Peptide not found: TQRKWEAARAAEQQR
Peptide not found: YDGKDYIALNEDLSS
Peptide not found: EDLRSWTAADMAAQT
Peptide not found: KSKFHDLCDFFLVIL
Peptide not found: GPLQLRALRQQDLPS


In [13]:
# randomly sample n kmers from input text
def sample_kmers(k, n, text):
    out = []
    indices = random.sample(range(len(text) - k + 1), k=n)
    for i in indices:
        substr = text[i:i+k]
        out.append(substr)
    return out

def generate_decoys(peptide, peptide_to_doc, num_decoys=10):
    if peptide not in peptide_to_doc:
        return
    transcripts = peptide_to_doc[peptide]
    if not transcripts:
        return
    longest_transcript = max(transcripts, key=lambda x: len(x.text))
    longest_transcript = longest_transcript.text.split('|')[1]
    decoys = sample_kmers(k=len(peptide), n=num_decoys, text=longest_transcript)
    for decoy in decoys:
        if decoy == peptide:
            print('Oops accidentally generated the hit: %s' % peptide)
        else:
            yield decoy
           
# make a bunch of decoy peptides
decoys = []
for hit in use_train_ms_df.peptide:
    decoys.extend(generate_decoys(hit, peptide_to_doc))

Oops accidentally generated the hit: LDRLLDLPAAASSED
Oops accidentally generated the hit: LAGEFIRASGVEARQ
Oops accidentally generated the hit: HSQVTPLSSQSVVIH
Oops accidentally generated the hit: SPNIVIALAGNKADL
Oops accidentally generated the hit: KEQEIERLNKLLRQH
Oops accidentally generated the hit: IPAILFLPRKRSQAE
Oops accidentally generated the hit: LGHPDTLNQGEFKEL
Oops accidentally generated the hit: MGEIASFDKAKLKKT
Oops accidentally generated the hit: IRLPYTASSGLMAPR
Oops accidentally generated the hit: RNRLFQENSVLSSLP
Oops accidentally generated the hit: EEFKRYEMLKEHERR
Oops accidentally generated the hit: QVLFETEISRKLFDT
Oops accidentally generated the hit: SSHICDFIRKTLNAG
Oops accidentally generated the hit: NEEGFFSARGHRPLD
Oops accidentally generated the hit: IDNKGIDSDASYPYK
Oops accidentally generated the hit: RRLSYNTASNKTRLS
Oops accidentally generated the hit: NPEPWNKLGPNDQYK
Oops accidentally generated the hit: VLYHYVAVNNPKKQE
Oops accidentally generated the hit: ITPMMELKP

In [16]:
# filter hits and decoys for any selenocysteines (U)
new_hits = [x for x in use_train_ms_df.peptide if 'U' not in x]
new_decoys = [x for x in decoys if 'U' not in x]

train_encoder = EncodableSequences.create(new_hits + new_decoys)
train_feature_array = train_encoder.variable_length_to_fixed_length_vector_encoding('BLOSUM62')

In [18]:
# since I'm making a logistic regression model, each peptide needs to be flattened to be a single numeric vector.
X_train = [x.flatten() for x in train_feature_array]
y_train = [1] * len(new_hits) + [0] * len(new_decoys)

In [22]:
# make an evaluation dataset

curated_df = pd.read_csv(get_path("data_curated", "curated_training_data.csv.bz2"))
curated_df.head()

,allele,peptide,measurement_value,measurement_inequality,measurement_type,measurement_kind,measurement_source,original_allele
0,BoLA-DRB3*001:01,AYAAQGYKVLVLNPSVAA,1541.0,=,quantitative,affinity,Walker - purified MHC/competitive/radioactivity,BoLA-DRB3*001:01
1,BoLA-DRB3*001:01,CGKYLFNWAVRTKLKLTPIA,8776.0,=,quantitative,affinity,Walker - purified MHC/competitive/radioactivity,BoLA-DRB3*001:01
2,BoLA-DRB3*001:01,ENLPYLVAYQATVCARAQAP,36805.0,=,quantitative,affinity,Walker - purified MHC/competitive/radioactivity,BoLA-DRB3*001:01
3,BoLA-DRB3*001:01,GIQYLAGLSTLPGNPAIASL,100000.0,>,quantitative,affinity,Walker - purified MHC/competitive/radioactivity,BoLA-DRB3*001:01
4,BoLA-DRB3*001:01,KGGRKPARLIVFPDLGVRVC,3336.0,=,quantitative,affinity,Walker - purified MHC/competitive/radioactivity,BoLA-DRB3*001:01


In [24]:
new_hits_set = set(new_hits)
new_hits_set.update(set(new_decoys))

# restrict validation set to 15mers binding DRB0101 that are not in the training data
affinity_df = curated_df.loc[
    (curated_df.measurement_kind == 'affinity') &
    (curated_df.allele == 'HLA-DRB1*01:01') &
    (curated_df['peptide'].str.len() == 15) &
    (~curated_df.peptide.isin(new_hits_set))
]
affinity_df.shape

(7988, 8)

In [60]:
# a row is a hit if it's <500nM
def is_hit(row):
    sign = row.measurement_inequality
    ic50 = row.measurement_value
    return sign in ['=', '<'] and ic50 < 500

affinity_df['hit'] = affinity_df.apply(is_hit, axis=1)

/Users/julia/miniconda3/envs/deep-learning-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [61]:
test_encoder = EncodableSequences.create(affinity_df.peptide)
test_feature_array = test_encoder.variable_length_to_fixed_length_vector_encoding('BLOSUM62')
X_test = [x.flatten() for x in test_feature_array]
y_test = affinity_df.hit

In [103]:
def run_model(model):
    print('Fitting...')
    model.fit(X_train, y_train)
    print('Done.')
    y_pred = model.predict_proba(X_test)
    return metrics.roc_auc_score(y_test, [x[1] for x in y_pred])

In [104]:
# try logistic regression model first
logreg = LogisticRegression(class_weight='balanced', max_iter=500)
run_model(logreg)

Fitting...


/Users/julia/miniconda3/envs/deep-learning-nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Done.


0.6006645890584387

In [108]:
clf = svm.SVC(kernel='linear', C=0.025, verbose=True, probability=True)
run_model(clf)

Fitting...
[LibSVM]Done.


0.5852566587653657

In [106]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, n_estimators=30, max_features=100, verbose=True)
run_model(clf)

Fitting...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Done.


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    4.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished


0.6294617912254074

In [111]:
# weighted classes in logistic regression
logreg = LogisticRegression(class_weight={0: 1, 1: 10})
run_model(logreg)

Fitting...


/Users/julia/miniconda3/envs/deep-learning-nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Done.


0.6004259049183096

In [71]:
alleles = [normalize_allele_name("HLA-DRB1*01:01")]
predictor = NetMHCIIpan(alleles=alleles, default_peptide_lengths=[15])

2020-12-30 16:21:29,379 - mhctools.netmhcii_pan - INFO - Using NetMHCIIpan 4.0
2020-12-30 16:21:29,648 - mhctools.base_commandline_predictor - INFO - Skipping allele DRB5_0108N: The suffix 'N' of 'DRB5*0108N' was not parsed
2020-12-30 16:21:30,246 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB30101	BoLA-DRA-DRB30101: Allele has too many parts: DRA-DRB30101	BoLA-DRA-DRB30101
2020-12-30 16:21:30,250 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB31101	BoLA-DRA-DRB31101: Allele has too many parts: DRA-DRB31101	BoLA-DRA-DRB31101
2020-12-30 16:21:30,253 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB31501	BoLA-DRA-DRB31501: Allele has too many parts: DRA-DRB31501	BoLA-DRA-DRB31501


In [77]:
training_peptides = new_hits + new_decoys
type(training_peptides)

list

In [78]:
netmhciipan_results = predictor.predict_subsequences(training_peptides)

2020-12-30 16:25:14,947 - mhctools.process_helpers - INFO - Ran 1 commands in 98.3112 seconds


In [80]:
netmhciipan_test_results = predictor.predict_subsequences(affinity_df.peptide)

2020-12-30 16:27:15,659 - mhctools.process_helpers - INFO - Ran 1 commands in 67.2750 seconds


In [81]:
# convert results to a vector of bools representing hits
affinity_df.peptide[:10]

198287    AAAAGWQTLSAALDA
198293    AAAGAEAGKATTEEQ
198295    AAAGLAAAAPLESRQ
198296    AAAGLAAAAPLESRQ
198303    AAALHNVKCKTPTQL
198306    AAASVPAADKFKTFE
198307    AAATATATAAVGAAT
198311    AAAYFVGYLKPTTFM
198313    AADHAAPEDKYEAFV
198314    AADHCPVVEVNGVTI
Name: peptide, dtype: object

In [112]:
y_pred = [-1 * x.percentile_rank for x in netmhciipan_test_results]
metrics.roc_auc_score(y_test, y_pred)

0.7121903078646317

In [114]:
alleles = [normalize_allele_name("HLA-DRB1*01:01")]
ba_predictor = NetMHCIIpan4_BA(alleles=alleles, default_peptide_lengths=[15])
netmhciipan_ba_test_results = ba_predictor.predict_subsequences(affinity_df.peptide)

2020-12-30 17:02:14,267 - mhctools.base_commandline_predictor - INFO - Skipping allele DRB5_0108N: The suffix 'N' of 'DRB5*0108N' was not parsed
2020-12-30 17:02:14,276 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB30101	BoLA-DRA-DRB30101: Allele has too many parts: DRA-DRB30101	BoLA-DRA-DRB30101
2020-12-30 17:02:14,278 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB31101	BoLA-DRA-DRB31101: Allele has too many parts: DRA-DRB31101	BoLA-DRA-DRB31101
2020-12-30 17:02:14,281 - mhctools.base_commandline_predictor - INFO - Skipping allele BoLA-DRA-DRB31501	BoLA-DRA-DRB31501: Allele has too many parts: DRA-DRB31501	BoLA-DRA-DRB31501
2020-12-30 17:03:26,879 - mhctools.process_helpers - INFO - Ran 1 commands in 72.4326 seconds


In [115]:
y_pred = [-1 * x.affinity for x in netmhciipan_ba_test_results]
metrics.roc_auc_score(y_test, y_pred)

0.8484445833649243

# Try to make a CNN

In [235]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [270]:
model = models.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(15, 21)))
model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
print(model.summary())

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

X_train = train_feature_array.copy()
y_train = np.array([1] * len(new_hits) + [0] * len(new_decoys))

X_test = test_feature_array.copy()
y_test = np.array(affinity_df.hit)

es = EarlyStopping(patience=10)
history = model.fit(X_train, y_train, epochs=1000, batch_size=50,
                    validation_data=(X_test, y_test), callbacks=[es])

out = model.predict(X_test)
metrics.roc_auc_score(y_test, out)

Model: "sequential_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_95 (Conv1D)           (None, 13, 32)            2048      
_________________________________________________________________
conv1d_96 (Conv1D)           (None, 11, 32)            3104      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 5, 32)             0         
_________________________________________________________________
conv1d_97 (Conv1D)           (None, 3, 64)             6208      
_________________________________________________________________
conv1d_98 (Conv1D)           (None, 1, 64)             12352     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 64)                0         
_________________________________________________________________
flatten_56 (Flatten)         (None, 64)              

0.6294631144668196

0.6226768887661811